In [2]:
import gudhi
import numpy as np

from functions.data_loading import get_filtered_data_range, get_data_range, get_flight_persistence
from functions.data_filtering import complete_flight_filter

from datetime import datetime
from collections import Counter
from tqdm import tqdm
from traffic.core import Traffic, Flight

In [5]:
origin = "bergen"
destination = "oslo"

unfiltered_data, file_name = get_data_range(origin, destination, datetime(year=2023, month=1, day=1), datetime(year=2024, month=1, day=1))
filtered_flights, file_name = get_filtered_data_range(unfiltered_data, file_name, complete_flight_filter(origin, destination))
persistences, _ = get_flight_persistence(filtered_flights, file_name)

data/complete_flights/bergen-oslo-2023-01-01-2024-01-01.pkl


In [11]:
circly_flights=[]
for i, tree in enumerate(persistences):
    persistence = tree.persistence()

    more_than_00001 = len([x for x in persistence if  x[0] == 1 and x[1][1] - x[1][0] > 0.0001])
    #more_than_00005 = len([x for x in persistence if  x[0] == 1 and x[1][1] - x[1][0] > 0.0005])
    if more_than_00001 > 0:
        circly_flights.append(i)

In [4]:
"""
codes = Counter()
for flight in circly_flights:
    codes.update(filtered_flights[flight].squawk)

codes
"""

'\ncodes = Counter()\nfor flight in circly_flights:\n    codes.update(filtered_flights[flight].squawk)\n\ncodes\n'

In [5]:
bool_array = np.zeros(len(persistences))
for i in circly_flights:
    bool_array[i] = 1

from functions.data_filtering import filter_by_bools, filter_flights
loopy_flights = filter_flights(filter_by_bools(bool_array), filtered_flights)

loopy_flights.map_leaflet()

Map(center=[60.30966211197027, 8.530049425198623], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [6]:
loopy_flights[38].map_leaflet()

Map(center=[60.37987787859019, 8.593328916812911], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [ ]:
flight_amount = len(persistences)
distance_matrix = np.zeros((flight_amount, flight_amount), dtype=float)

for i in tqdm(range(197, flight_amount)):
    for j in range(i+1, flight_amount):
        print(i, j)
        tree_i = persistences[i]
        tree_j = persistences[j]
        pers_i = [x for x in tree_i.persistence_intervals_in_dimension(1) if  x[1] - x[0] > 0.0001]
        pers_j = [x for x in tree_j.persistence_intervals_in_dimension(1) if  x[1] - x[0] > 0.0001]
        dist = gudhi.bottleneck_distance(pers_i, pers_j, 0.001)
        distance_matrix[i, j] = dist
        distance_matrix[j, i] = dist

  0%|          | 0/3022 [00:00<?, ?it/s]

197 198
197 199
197 200
197 201
197 202
197 203
197 204
197 205
197 206
197 207
197 208
197 209
197 210
197 211
197 212
197 213
197 214
197 215
197 216
197 217
197 218
197 219
197 220
197 221
197 222
197 223
197 224
197 225
197 226
197 227
197 228
197 229
197 230
197 231
197 232
197 233
197 234
197 235
197 236
197 237
197 238
197 239
197 240
197 241
197 242
197 243
197 244
197 245
197 246
197 247
197 248
197 249
197 250
197 251
197 252
197 253
197 254
197 255
197 256
197 257
197 258
197 259
197 260
197 261
197 262
197 263
197 264
197 265
197 266
197 267
197 268
197 269
197 270
197 271
197 272
197 273
197 274
197 275
197 276
197 277
197 278
197 279
197 280
197 281
197 282
197 283
197 284
197 285
197 286
197 287
197 288
197 289
197 290
197 291
197 292
197 293
197 294
197 295
197 296
197 297
197 298
197 299
197 300
197 301
197 302
197 303
197 304
197 305
197 306
197 307
197 308
197 309
197 310
197 311
197 312
197 313
197 314
197 315
197 316
197 317
197 318
197 319
197 320
197 321
197 322


In [5]:
print(sum(sum(distance_matrix)))

0.0


In [8]:
distance_matrix.nonzero()

(array([], dtype=int64), array([], dtype=int64))

In [1]:
pers_i = persistences[197].persistence_intervals_in_dimension(1)
pers_j = persistences[448].persistence_intervals_in_dimension(1)
dist = gudhi.bottleneck_distance(pers_i, pers_j)
dist

NameError: name 'persistences' is not defined

In [12]:
pers_i = [x for x in persistences[197].persistence_intervals_in_dimension(1) if  x[1] - x[0] > 0.0001]
pers_j = [x for x in persistences[448].persistence_intervals_in_dimension(1) if  x[1] - x[0] > 0.0001]
print(pers_i)
print(pers_j)

[array([8.33126906e-07, 1.26088065e-04]), array([8.82459141e-05, 7.50946700e-03])]
[array([0.00016188, 0.00220959]), array([5.88552650e-05, 2.50344004e-03])]


In [ ]:
print(gudhi.bottleneck_distance(pers_i, pers_j, 0.001))

In [7]:
print(persistences[98].persistence_intervals_in_dimension(1))

print

[[0.00829266 0.00829277]
 [0.00829325 0.00829325]
 [0.00829328 0.00829335]
 [0.00829229 0.00833566]]


In [32]:
persistences[circly_flights[0]].persistence_intervals_in_dimension(1)

IndexError: list index out of range

In [21]:
sum(sum(distance_matrix))

0.0